In [33]:
import pandas as pd

dataFrame = pd.read_csv('akiyo_25_first.csv')

print(dataFrame['class'].unique())
for i in range(0, len(dataFrame['class'])):
    for p in dataFrame.keys():
        print(f'elemento {i:2}, parametro {p:20}: {dataFrame.loc[i, p]:40}')
    print('*-'*40)


for i in range(0, len(dataFrame['class'])):
    for p in range(0, len(dataFrame.keys())):
        print(f'elemento {i:2}, parametro {dataFrame.keys()[p]:20}: {dataFrame.iloc[i, p]:40}')
    print('*-'*40)

['AlfCovariance' 'IntraPrediction' 'DQIntern' 'void DQIntern'
 'unsigned long RdCost' 'void InterpolationFilter' 'InterPrediction'
 'CodingStructure' 'no_class' 'QuantRDOQ' 'RdCost' 'EncCu' 'IntraSearch'
 'TrQuant' 'Picture' 'MatrixIntraPrediction']
elemento  0, parametro percentageTime      :                                     5.88
elemento  0, parametro cumulativeTime      :                                     3.02
elemento  0, parametro selfSeconds         :                                     3.02
elemento  0, parametro calls               :                                5798885.0
elemento  0, parametro selfMs/call         :                                      0.0
elemento  0, parametro totalMs/call        :                                      0.0
elemento  0, parametro function            : AlfCovariance::gnsCholeskyDec           
elemento  0, parametro class               : AlfCovariance                           
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-